### Deploy Grafana dashboard

In [1]:
import nuclio
import os

In [2]:
%run set_env.ipynb

In [3]:
# nuclio: start-code

In [4]:
from grafwiz import *
import os
import v3io_frames as v3f

In [5]:
def handler(context, event):
    
    raw_stream_url = f"https://{os.getenv('API_GATEWAY')}-{os.getenv('PROJECT')}.{os.getenv('IGZ_NAMESPACE_DOMAIN')}/?shardId={os.getenv('shardId')}&container={os.getenv('IGZ_CONTAINER')}&stream={os.getenv('RAW_VIDEO_STREAM')}"
    tagged_stream_url = f"https://{os.getenv('API_GATEWAY')}-{os.getenv('PROJECT')}.{os.getenv('IGZ_NAMESPACE_DOMAIN')}/?shardId={os.getenv('shardId')}&container={os.getenv('IGZ_CONTAINER')}&stream={os.getenv('TAGGED_VIDEO_STREAM')}"
    
    params_js = """{
         from:ctrl.range.from.format('x'),  // x is unix ms timestamp
         to:ctrl.range.to.format('x'), 
         height:ctrl.height,
         now:Date.now(),
         interval: ctrl.template('$__interval'),
         sample: 'Not escaped: $__interval',
         since:ctrl.lastRequestTime
        }"""
    
    dash = Dashboard("Stream Viewer Test", start='now', dataSource='iguazio', refresh='1s')
    
    raw_stream = Ajax(title='Raw Stream',
                      url=raw_stream_url,
                      method="iframe",
                      params_js=params_js)
    
    tagged_stream = Ajax(title='Tagged Stream',
                      url=tagged_stream_url,
                      method="iframe",
                      params_js=params_js)
    
    dash.row([raw_stream, tagged_stream])
    
    return dash.deploy('http://grafana')

In [6]:
# nuclio: end-code

In [7]:
# nuclio: ignore
# converts the notebook code to deployable function with configurations
from mlrun import code_to_function, mount_v3io, mlconf
import os

fn = code_to_function('grafana-dashboard', kind='nuclio')

In [8]:
# set the API/trigger, attach the home dir to the function
fn.with_http(workers=1).apply(mount_v3io())
#fn.spec.build.base_image = 'mlrun/mlrun'
fn.spec.base_spec['spec']['build']['baseImage'] = 'python:3.6-jessie'
fn.spec.build.commands = ['pip install git+https://github.com/v3io/grafwiz v3io_frames']
fn.spec.min_replicas = 1
fn.spec.max_replicas = 1
fn.set_env('API_GATEWAY', os.getenv('API_GATEWAY'))
fn.set_env('PROJECT', os.getenv('PROJECT'))
fn.set_env('IGZ_NAMESPACE_DOMAIN', os.getenv('IGZ_NAMESPACE_DOMAIN'))
fn.set_env('RAW_VIDEO_STREAM' ,os.getenv('RAW_VIDEO_STREAM'))
fn.set_env('TAGGED_VIDEO_STREAM', os.getenv('TAGGED_VIDEO_STREAM'))
fn.set_env('IGZ_CONTAINER', os.getenv('IGZ_CONTAINER'))
fn.set_env('shardId', os.getenv('shardId'))
fn.apply(mount_v3io())

In [9]:
# print(fn.to_yaml())

In [10]:
# nuclio: ignore
addr = fn.deploy(project='objectrecognition')

> 2020-10-26 16:38:21,000 [info] deploy started
[nuclio] 2020-10-26 16:38:22,082 (info) Build complete
[nuclio] 2020-10-26 16:38:28,145 (info) Function deploy complete
[nuclio] 2020-10-26 16:38:28,152 done creating objectrecognition-grafana-dashboard, function address: 18.224.192.183:32527


In [11]:
!curl $addr